In [1]:
using JuMP, Ipopt, Plots, Printf, LinearAlgebra, SCS

In [2]:
N = 3
λ = 1
ni = 4 * (N + 1)^2
np = 5 * N + 1
ns  = ni - 12
nt = np + ns + 1
println(ni)
println(np)
println(ns)
println(nt)


64
16
52
69


In [3]:
69*69

4761

We Define the states now

In [4]:
S = Dict()
T = []
TG = []
TB = []
for ua in 0:N
    for ub in 0:N
        for sa in 0:1
            for sb in 0:1
                index = (2 * sa + sb) * (N + 1)^2 + (N + 1) * ua + ub
                S[index] = (ua, ub, sa, sb)
            end
        end
    end
end
Skeyer = Dict(value => key for (key, value) in S);


Define Terminal states

In [5]:
# Initialize lists
T = []
TG = []
TB = []

# Loop through the ranges
for ua in 0:N
    for ub in 0:N
        for sa in 0:1
            for sb in 0:1
                i = (2 * sa + sb) * (N + 1)^2 + (N + 1) * ua + ub
                if ua == N && ub == 0
                    push!(TG, i)
                    push!(T, i)
                end
                if ua == 0 && ub == N
                    push!(TG, i)
                    push!(T, i)
                end
                if ua == N && ub == N
                    push!(TB, i)
                    push!(T, i)
                end
                S[i] = (ua, ub, sa, sb)
            end
        end
    end
end
Tc = [i for i in 0:ni-1 if i ∉ T]

sort!(T);
sort!(TG);
sort!(TB);


println("Terminal States")
println("T: ", T)
println("TG: ", TG)
println("TB: ", TB)


Terminal States
T: Any[3, 12, 15, 19, 28, 31, 35, 44, 47, 51, 60, 63]
TG: Any[3, 12, 19, 28, 35, 44, 51, 60]
TB: Any[15, 31, 47, 63]


Parameters

In [6]:
f⁺ = collect([[N * (j-1) + i for j in 1:2] for i in 1:N]);
# f⁺ = collect([[rand() for j in 1:2] for i in 1:N])
f⁺=reduce(hcat,f⁺)
f⁻ = collect([[N * (j-1) + i + (2 * N) for j in 1:2] for i in 1:N]);
f⁻=reduce(hcat,f⁻)
f⁺_flattened = collect(Iterators.flatten(f⁺'))
f⁻_flattened = collect(Iterators.flatten(f⁻'))
g = [4 * N  + i for i in 1:N];
koff = 5 * N + 1
P = collect(vcat(f⁺_flattened, f⁻_flattened, g, [koff]))
print(P)


[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]

Defining matrices

In [7]:
M=zeros(Int, ni, ni, np);
for (u, u_) in Iterators.product(values(S), values(S))
    # Skip if u and u_ are the same
    if u == u_
        continue
    end

    i = Skeyer[u]
    j = Skeyer[u_]
    flag = 0
    ua, ub, sa, sb = u
    ua_, ub_, sa_, sb_ = u_

    tempk = 0

    if j == i + (N + 1) && ub == ub_ && sa == sa_ && sb == sb_
        flag = 1
        tempk = N * sa + ua + 1
    elseif j == i - (N + 1) && ub == ub_ && sa == sa_ && sb == sb_
        flag = 1
        tempk = sa * N + ua + 2 * N 
    elseif j == i + 1 && ua == ua_ && sa == sa_ && sb == sb_
        flag = 1
        tempk = N * sb + ub + 1
    elseif j == i - 1 && ua == ua_ && sa == sa_ && sb == sb_
        flag = 1
        tempk = sb * N + ub + 2 * N 
    elseif j == i + 2 * (N + 1)^2 && ua == ua_ && ub == ub_ && ub != 0 && sb == sb_
        flag = 1
        tempk = 4 * N + ub 
    elseif j == i - 2 * (N + 1)^2 && ua == ua_ && ub == ub_ && sb == sb_
        flag = 1
        tempk = 5 * N + 1
    elseif j == i + (N + 1)^2 && ua == ua_ && ub == ub_ && ua != 0 && sa == sa_
        flag = 1
        tempk = 4 * N + ua 
    elseif j == i - (N + 1)^2 && ua == ua_ && ub == ub_ && sa == sa_
        flag = 1
        tempk = 5 * N + 1
    end

    if flag == 1
        M[i+1, j+1, tempk] = flag
    end
end
for k in 1:np
    # Calculate the sum of each row for the kth slice of M (equivalent to M[:,:,k] in Python)
    M_i = [sum(M[:, :, k][i, :]) for i in 1:ni]

    for u in values(S)
        i = Skeyer[u]
        M[i+1, i+1, k] = -M_i[i+1]
    end
end

In [8]:
P_=reshape(P, np, 1)
Q = reduce((x, y) -> x + y, [P_[k] * M[:, :, k] for k in 1:np])
# println(Q)
# for row in eachrow(Q)
#     for elem in row
#         @printf("%.4f\t", elem)
#     end
#     println()
# end

64×64 Matrix{Int64}:
 -2    1    0    0    1    0    0    0  …    0    0    0    0    0    0    0
  7  -23    2    0    0    1    0    0       0    0    0    0    0    0    0
  0    8  -26    3    0    0    1    0       0    0    0    0    0    0    0
  0    0    9  -25    0    0    0    1       0    0    0    0    0    0    0
  7    0    0    0  -23    1    0    0       0    0    0    0    0    0    0
  0    7    0    0    7  -44    2    0  …    0    0    0    0    0    0    0
  0    0    7    0    0    8  -47    3       0    0    0    0    0    0    0
  0    0    0    7    0    0    9  -46       0    0    0    0    0    0    0
  0    0    0    0    8    0    0    0       0    0    0    0    0    0    0
  0    0    0    0    0    8    0    0       0    0    0    0    0    0    0
  0    0    0    0    0    0    8    0  …    0    0    0    0    0    0    0
  0    0    0    0    0    0    0    8       0    0    0    0    0    0    0
  0    0    0    0    0    0    0    0       0    0    

In [9]:
R = zeros(ns, ni)
for i in 1:ns
    R[i,Tc[i]+1] = 1
end
Qtilde = R * Q * R'

52×52 Matrix{Float64}:
 -2.0    1.0    0.0    1.0    0.0  …    0.0    0.0    0.0    0.0    0.0
  7.0  -23.0    2.0    0.0    1.0       0.0    0.0    0.0    0.0    0.0
  0.0    8.0  -26.0    0.0    0.0       0.0    0.0    0.0    0.0    0.0
  7.0    0.0    0.0  -23.0    1.0       0.0    0.0    0.0    0.0    0.0
  0.0    7.0    0.0    7.0  -44.0       0.0    0.0    0.0    0.0    0.0
  0.0    0.0    7.0    0.0    8.0  …    0.0    0.0    0.0    0.0    0.0
  0.0    0.0    0.0    0.0    0.0       0.0    0.0    0.0    0.0    0.0
  0.0    0.0    0.0    8.0    0.0       0.0    0.0    0.0    0.0    0.0
  0.0    0.0    0.0    0.0    8.0       0.0    0.0    0.0    0.0    0.0
  0.0    0.0    0.0    0.0    0.0       0.0    0.0    0.0    0.0    0.0
  0.0    0.0    0.0    0.0    0.0  …    0.0    0.0    0.0    0.0    0.0
  0.0    0.0    0.0    0.0    0.0       0.0    0.0    0.0    0.0    0.0
  0.0    0.0    0.0    0.0    0.0       0.0    0.0    0.0    0.0    0.0
  ⋮                                ⋱     

In [10]:
Mtilde = zeros(ns, ns, np)
for k in 1:np
    Mtilde[:, :, k] = R * M[:, :, k] * R'
end

In [11]:
# Mtilde = zeros(ns, ns, np)
# for i in 1:ns
#     for j in 1:ns
#         for k in 1:np
#             Mtilde[i, j, k] = sum(R[i, a] * R[j, b] * M[a, b, k] for a in 1:ni, b in 1:ni)
#         end
#     end
# end

In [12]:
eis = Matrix{Float64}(I, ns, ns)
Ai = zeros(ns, np, ns)
for k in 1:ns
    ei=eis[k,:]
    for i in 1:ns
        for j in 1:np
            Ai[i,j,k]=ei'* Mtilde[:,i,j]   
        end 
    end
end

In [13]:
alpha=zeros(np,ns)
for i in 1:ns
    Si=Tc[i]
    for j in 1:ni
        for k in 1:np
            if (j-1 ∈ TB)
                alpha[k,i]+=M[Si+1,j,k]
            end
            
        end
    end
end

In [14]:
Di=zeros(nt,nt,ns)
for i in 1:ns
    Di[np+1:np+ns,1:np,i]=Ai[:,:,i]
    Di[1:np,nt,i]=λ*alpha[:,i]'
    Di[nt,nt,i]=1
end
# for row in eachrow(Di[:, :, end])
#     println(row)
# end

In [15]:
Ei=zeros(nt,nt,nt)
for i in 1:nt
    Ei[i,nt,i]=1
end
S0=Ei[:,:,np+1]

69×69 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  

In [16]:
#Symmetrise the matrices
D_i=copy(Di)
for i in 1:ns
    D_i[ :, :,i] = (Di[ :, :,i] + Di[ :, :,i]') / 2
end
        
S_0 = (S0 + S0') / 2
E_i = copy(Ei)
for i in 1:nt
    E_i[ :, :,i] = (Ei[ :, :,i] + Ei[ :, :,i]') / 2
end

In [17]:
Cis=zeros(nt,nt,np)
for i in 1:np
    Cis[i,i,i]=1
end

We have defined all our matrices! We can move towards formulating a SDP problem for our case